Finding Stabilizer States using Gates' Symplectic Forms
===

### The Formula

The formula we are going to be working with is the following:

$$\rho_{\left|\psi\right\rangle} = \left|\psi\right\rangle \left\langle\psi\right| = \frac{1}{2^n} \prod^{n}_{i=1} \left(\mathbf{I} + g_i\right)$$

where:
* $\rho_{\left|\psi\right\rangle}$ is the density matrix of the quantum state $\left|\psi\right\rangle$
* $n$ is the number of qubits we are working with
* $\mathbf{I}$ is the identity matrix, which is of size $2^n\times 2^n$
* $g_i$ are the generators of size $2^n\times 2^n$, of the stabilizer group $S\subseteq P_{n}$
* $P_n$ is the Pauli group on n qubits

---
### Functions Manipulating Gates in Pauli and Symplectic Form

The ability to convert gates from their Pauli representation to their symplectic form is very useful.

The base gates and their associated matrices are as follows:

$$X = \begin{pmatrix}0&1\\1&0\end{pmatrix} \;\; Z = \begin{pmatrix}1&0\\0&-1\end{pmatrix} \;\; Y = \begin{pmatrix}0&-i\\i&0\end{pmatrix}$$

The Pauli representation is the gates' most typical form. An example of such representation is $X \otimes I \otimes Z \otimes Y$. This shows that an $X$ gate acts on the first qubit, the second qubit is unchanged, a $Z$ gate acts on the third qubit, and $Y$ acts on the fourth qubit. Usually when represented in this way the tensor product notation is collapsed on, such that the gates would be instead written as $XIZY$. Noting that $(X)(Z) = iY$, these gates may also be represented by the following equation:

$$\mathcal{P} = (-1)^c(-i)^{\vec{q}\cdot\vec{p}}(X)^\vec{q}(Z)^\vec{p}$$

The symplectic representation of quantum gates is another way of describing whether an X and/or Z gate is acting on a qubit. It uses the equation above and hilights just the vectors $\vec{q},\vec{p}$ and $c$. For instance the example gate above may be rewritten in its symplectic form as:

$$XIZY \rightarrow \left(--\vec{q}--|--\vec{p}--|c\right) = \left(1,0,0,1|0,0,1,1|0\right)$$

The advantage of this representation is multiplication of gates becomes addition of their symplectic binary strings:
$$\begin{align*}
(X ⊗ I ⊗ Z ⊗ Y )(−X ⊗ Y ⊗ X ⊗ X) &= [1, 0, 0, 1|0, 0, 1, 1|0]\\
&+ [1, 1, 1, 1|0, 1, 0, 0|1]\\
&= [0, 1, 1, 0|0, 1, 1, 1|1]\\
&= −I ⊗ Y ⊗ Y ⊗ Z
\end{align*}$$

It should be noted that the addition is only perfect if the twisted inner product of the two gates is equal to zero:

$$g_1 \odot g_2 = \vec{q}_1\cdot\vec{p}_2 \oplus \vec{q}_2\cdot\vec{p}_1 = 0$$

This ensures there are no unwanted imaginary phases, and hence why we may regard the phase bit as only being binary and not quaternary.

With that information in mind, we needed to create some functions that could do the following:
* Convert a gate from its Pauli representation to its symplectic form
* Multiply two sumplectic gates together
* Keep the phases of the multiplied gates in check
* Convert a gate from its symplectic form to its Pauli representation

---

### This Notebook

We have in previous notebooks managed to show the possible symplectic combinations for $n=2$ is $60$, using the following rules:

* The combination must contain gates that are linearly independant (in their symplectic forms). This means $g_i \neq \mathbf{I}$, and $g_i \neq \pm g_j$
* The twisted inner product of the gates' symplectic forms must equal zero mod 2; $g_i \odot g_j = 0 \;\mathbb{Z}_2 \;,\;\forall i,j\in \{1,2,...,n\}\;,\;i\neq j$ 
* The set of multiples of gates possible through the equation $\prod^{n}_{i=1} \left(\mathbf{I} + g_i\right)$ are unique, and also are all linearly independant.

We will attempt to count the possible unique gates that will form a symplectic basis for $n=3$. We expect to find $1080$ once all the rules above have been followed

In [ ]:
import numpy as np
from numpy.linalg import eig
import itertools as it

In [ ]:
#this function converts a gate from Pauli form to symplectic
#it expects the gate in a string form
def symplectify(gate):
    gate = list(gate)
    
    minus=False
    imag =False
    
    if "-" in gate:               #checks if the gate has a minus phase
        index = gate.index("-")   #also removes the minus such that the symplectic form is the right shape
        gate.pop(index)
        minus=True
    if "i" in gate:               #checks if the gate has an imaginary phase
        index = gate.index("i")   #also removes the imaginary number such that the symplectic form is the right shape
        gate.pop(index)
        imag=True
    
    form = [np.zeros(len(gate)),np.zeros(len(gate)),np.array([0])]        #set up the shape of the symplectic form
    
    if minus == True:               #this governs the phase bit of the symplectic form
        form[2][0] += 1
    if imag == True:                #later on our gates should never have an imaginary phase, -
        print("Unwanted phase")     #- so if we have one we have an issue
    
    for S in gate:                  #here we run over every tensored gate
        if S == "X":                #if the gate is an "X", the symplectic form's first column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1

        if S == "Z":                #if the gate is a "Z", the symplectic form's second column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[1][index] += 1

        if S == "Y":                #if the gate is a "Y", the symplectic form's first and second columns are updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1
            form[1][index] += 1
            
    return form                     #returns the gate's symplectic form



#this function calculates the phase change produced when you multiply two gates together
def g_(a,b,c,d):
    phase=0
    for i in range(len(a)):
        phase += a[i]**2*b[i]**2 + c[i]**2*d[i]**2 - (a[i]+c[i])**2*(b[i]+d[i])**2 + 2*c[i]*b[i]
    return phase%4



#this function multiplies two gates together
#it expects the gates in the symplectic form
def symp_mul(gate1, gate2):
    new_gate = [np.zeros(len(gate1[0])),np.zeros(len(gate1[0])),np.array([0])]
    new_gate[0] = (gate1[0] + gate2[0])%2
    new_gate[1] = (gate1[1] + gate2[1])%2
    new_gate[2] = ((gate1[2]*2 + gate2[2]*2 + g_(gate1[0],gate1[1],gate2[0],gate2[1]))%4)/2
    
    return new_gate



#this function converts the gate from symplectic form to Pauli form
#it expects the gate to be in the symplectic form
def Paulify(symp_gate):
    Pgate = []
    for i in range(len(symp_gate[0])):
        if symp_gate[0][i] == 1 and symp_gate[1][i] == 1: #if theres a 1 in both q and p, the ith gate is Y
            Pgate.append("Y")
        elif symp_gate[0][i] == 1:                        #elif theres a 1 in q only, the ith gate is X
            Pgate.append("X")
        elif symp_gate[1][i] == 1:                        #elif theres a 1 in p only, the ith gate is Z
            Pgate.append("Z")
        else:                                             #else the gate must be I
            Pgate.append("I")
            
    Pgate = "".join(Pgate)
    
    if symp_gate[2][0]==1:                                #if the phase bit is 1 then the gate must be negative
        Pgate = "-" + Pgate
        
    return Pgate

In [ ]:
#an iteration function that has been repurposed, hence the misplaced function name
def Fnp(n,Fp):
    combi = list(list(it.product(Fp, repeat=n)))
    return combi

In [ ]:
#define all the gate matrices and names for use in the functions
I = np.array([[1,0],
              [0,1]])

X = np.array([[0,1],
              [1,0]])

Z = np.array([[1,0],
              [0,-1]])

Y = np.array([[0,-1j],
              [1j,0]])

base_gate_names = ["I", "X", "Y", "Z"]
base_gate_dict = {"I":I, "-I":-I, "X":X, "-X":-X, "Y":Y, "-Y":-Y, "Z":Z, "-Z":-Z}



#this function creates all possible tensor multiplied gate names for a given n
def gate_namemaker(n):
    gate_names = []
    for tup in Fnp(n,base_gate_names):
        name = ""
        for g in tup:
            name+=g
        gate_names.append(name)
        gate_names.append("-"+name)
        
    return gate_names



#this function creates all possible tensor multiplied gates in matrix form for a given n
def gate_matmaker(n):
    gate_mats = []
    for tup in Fnp(n,base_gate_names):

        prod1 = [1]
        prod2 = [-1]
        for g in tup:
            prod1 = np.kron(prod1,base_gate_dict[g])
            prod2 = np.kron(prod2,base_gate_dict[g])

        gate_mats.append(prod1)
        gate_mats.append(prod2)
    
    return gate_mats



#this function uses the above two functions to create a dictionary of the tensor multiplied gates names
#correspondng to the matrix form
def gate_dictmaker(n):
    gate_names = gate_namemaker(n)
    gate_mats = gate_matmaker(n)

    gate_dict = {}
    for i in range(len(gate_names)):
        gate_dict[gate_names[i]] = gate_mats[i]

    return gate_dict



#this funtion finds the set of stabilizer gates given a set of basis gates, excluding the Identity gate
def all_gates_mul(basis_gates):
    n = len(basis_gates)
    
    index_list = []                              #unique combinations of the basis gates must be generated for this to work
    for i in range(n):                           #find all the indexes of the gates that will give unique combinations
        for thing in [list(yoke) for yoke in Fnp(i+1,Fp(n))]:
            if len(thing) == len(set(thing)):    #iterate over the indices in a way that will not produce copies
                thing.sort()
                if thing not in index_list:
                    index_list.append(thing)
                    
    final_gate_list = []
    for indexes in index_list:                   #run over all unique sets of indices
        gates_list = []
        for index in indexes:                    #create the unique lists of basis gate combinations
            gates_list.append(basis_gates[index])
        gate_final = "I"*n                       #multiply all the gates in the combination together to generate the full -
        for gate in gates_list:                  #- set of stabilizer gates for the stabilizer state in question
            gate_final = Paulify(symp_mul(symplectify(gate_final), symplectify(gate)))
        final_gate_list.append(gate_final)
        
    return final_gate_list                       #return the set of stabilizer gates

In [ ]:
gate_dictmaker(2)

In [ ]:
#this function calculates the twisted inner product of two gates
#expects string form of gate
def twisted_prod2(g1,g2):
    g1 = symplectify(g1)
    g2 = symplectify(g2)
    prod = np.dot(g1[0],g2[1]) + np.dot(g1[1],g2[0])
    return prod%2

---

### The first attempt

We decided that we would first try to implement a similar method to that which we did for $n=2$. We realised that in order to generalise this we would need to be able to run a variable number of for loops, which we don't know how to do. We read around that it can be done with recursive coding, but it has been a few years since we have done anything like that and we weren't able. However, if someone were to code it, it would be a general way to find the combos for any value of $n$, given the proper amount of computing power.

As such we went with just coding the particular $n=3$ case. However, after a while of experimenting we found out the number of possible combinations of gates is in excess of two million, so we figured we had to take a different approach.

In [ ]:
#didnt work, far too many combos
#WARNING >2,000,000!!!!!
#dont bother running this
n = 3

gate_names = namemaker(n)
gate_mats = matmaker(n)
gate_dict = dictmaker(n)

combos = []

for g1 in range(len(gate_names)):
    print("g1 =",gate_names[g1])
    
    if gate_names[g1] == "I"*n:
        continue
    if gate_names[g1] == "-"+"I"*n:
        continue
        
    for g2 in range(len(gate_names)):
        print("g2 =",gate_names[g2])
        if gate_names[g2] == "I"*n:
            continue
        if gate_names[g2] == "-"+"I"*n:
            continue
        if gate_names[g1] == gate_names[g2]:
            continue
        if gate_names[g1] == "-"+gate_names[g2]:
            continue
        if gate_names[g2] == "-"+gate_names[g1]:
            continue
            
        if twisted_prod2(gate_names[g1],gate_names[g2]) == 1:
            continue
        
        
        
        for g3 in range(len(gate_names)):
            print("g3 =",gate_names[g3])
            if gate_names[g3] == "I"*n:
                continue
            if gate_names[g3] == "-"+"I"*n:
                continue
            if gate_names[g1] == gate_names[g3]:
                continue
            if gate_names[g2] == gate_names[g3]:
                continue
            
            if twisted_prod2(gate_names[g1],gate_names[g3]) == 1:
                continue
            if twisted_prod2(gate_names[g2],gate_names[g3]) == 1:
                continue
            
            combo = [gate_names[g1], gate_names[g2], gate_names[g3], 
                     Paulify(symp_mul(symplectify(gate_names[g1]), symplectify(gate_names[g2]))),
                     Paulify(symp_mul(symplectify(gate_names[g1]), symplectify(gate_names[g3]))),
                     Paulify(symp_mul(symplectify(gate_names[g2]), symplectify(gate_names[g3])))]
            
            combo.sort()
            
            if ("-II" or "II") in combo:
                continue
                
            if combo not in combos:
                combos.append(combo)
combos

---

### The Rule Breakdown Strategy

Our next strategy was to break the rules above into smaller and hopefully more easily runnable codeblocks. Our hope in doing this is to do far less computations in one go in the larger for loops, cut down the possible combinations we must still check early on, and get the more heavily computations to be done in smaller for loops. 

Starting off, we wanted to see exactly how many possible combinations there are for the gates. First it is important to know there are 128 total gates for $n=3$. The codeblock below finds all combinatinations by iterating over every gate in the gate list thrice, meaning we end up with $128^3 = 2,097,152$ combinations. 

In [ ]:
len(Fnp(3,gate_namemaker(3)))

---

We reasoned we don't need to consider the combinations with the identity matrix and its negative, and as they are the first and second in the list respectively, we simply removed them. This leaves us with $126^3 = 2,000,376$, already an improvement.

In [ ]:
gate_names = gate_namemaker(3)
print("Length =", len(gate_names))
print("Removed",gate_names.pop(0))
print("Removed",gate_names.pop(0))
print("Length =", len(gate_names))

In [ ]:
len(Fnp(3,gate_names))

---

The first rule we enforced is the linearity. We made sure none of the combinations with copies of gates or their negatives remained to be considered. With that we were left with $1,906,128$ combos. It takes a small amount of time, but it is a fairly computationally light calculation.

In [ ]:
combos = Fnp(3,gate_names)

In [ ]:
combos2 = []
for i in range(len(combos)):
    if (combos[i][0] == combos[i][1]) or (combos[i][0] == combos[i][2]) or (combos[i][1] == combos[i][2]):
        continue
    if (combos[i][0] == "-"+combos[i][1]) or (combos[i][1] == "-"+combos[i][0]):
        continue
    if (combos[i][0] == "-"+combos[i][2]) or (combos[i][2] == "-"+combos[i][0]):
        continue
    if (combos[i][1] == "-"+combos[i][2]) or (combos[i][2] == "-"+combos[i][1]):
        continue
    
    combos2.append(combos[i])
    
print("Done")

In [ ]:
len(combos2)

---

Next we enforced the second rule, that being that the twisted product of every gate in the combo mustn't be one. This prevents any unwanted extra phases in the gates when they are multiplied together. This step was a small bit more computationally heavy than the last, but managed to whittle the possible combos way down to $196,560$, which is an order of magnitude lower than the last number.

In [ ]:
combos3 = []
for i in range(len(combos2)):
    if twisted_prod2(combos2[i][0],combos2[i][1])==1:
        continue
    if twisted_prod2(combos2[i][1],combos2[i][2])==1:
        continue
    if twisted_prod2(combos2[i][0],combos2[i][2])==1:
        continue
    combos3.append(combos2[i])
    
print("Done")

In [ ]:
len(combos3)

---

Here is the beginning of my enforcing of the third rule, where we must multiply the gates together in every concievable way and make sure the set produced is unique among the other sets. For $n=3$ the equation we are using becomes:

$$\begin{eqnarray*}
\rho_{\left|\psi\right\rangle} = \left|\psi\right\rangle \left\langle\psi\right| &=& \frac{1}{2^3} \prod^{3}_{i=1} \left(\mathbf{I} + g_i\right)\\
&=& \frac{1}{8}\left(\mathbf{I} + g_1\right)\left(\mathbf{I} + g_2\right)\left(\mathbf{I} + g_3\right)\\
&=& \frac{1}{8}\left(\mathbf{I} + g_1 + g_2 + g_3 + g_1g_2 + g_1g_3 + g_2g_3 + g_1g_2g_3\right)\\
\end{eqnarray*}
$$

As such the we must be able to take the set $\{g_1,g_2,g_3\}$ and convert it to the set $\{g_1,g_2,g_3,g_1g_2,g_1g_3,g_2g_3,g_1g_2g_3\}$, which we must check is unique. 

The codeblock below will generate the possible sets $\{g_1,g_2,g_3,g_1g_2\}$ and remove the copies. This shortened set is because we figured creating the entire set of length 8 in one go would take a very long time, especially since we still have $196,560$ possible combos left. It should also be stated that when we are generating these sets it is important to make sure the order of the gates is preserved, or we may run into issues later on.

Running the codeblock below managed to get our list of combos down to $37,800$

In [ ]:
combos4 = []
combos4prime = []
for i in range(len(combos3)):
    combo = [combos3[i][0], combos3[i][1], combos3[i][2], 
             Paulify(symp_mul(symplectify(combos3[i][0]), symplectify(combos3[i][1])))]
    
    combo2 = combo.copy()
    combo2.sort()
    
    if tuple(combo2) not in combos4prime:
        combos4prime.append(tuple(combo2))
        combos4.append(tuple(combo))
        
print("Done")

In [ ]:
len(combos4)

---

Next we will generate the possible sets $\{g_1,g_2,g_3,g_1g_2,g_1g_3\}$ and remove the copies. This step reduces the list to $13,500$ 

In [ ]:
combos5 = []
combos5prime = []
for i in range(len(combos4)):
    combo = [combos4[i][0], combos4[i][1], combos4[i][2], combos4[i][3],
             Paulify(symp_mul(symplectify(combos4[i][0]), symplectify(combos4[i][2])))]
    
    combo2 = combo.copy()
    combo2.sort()
    
    if tuple(combo2) not in combos5prime:
        combos5prime.append(tuple(combo2))
        combos5.append(tuple(combo))
        
print("Done")

In [ ]:
len(combos5)

---

Generating the possible sets $\{g_1,g_2,g_3,g_1g_2,g_1g_3,g_2g_3\}$ and removing the copies reduces the list to $6,975$ 

In [ ]:
combos6 = []
combos6prime = []
for i in range(len(combos5)):
    combo = [combos5[i][0], combos5[i][1], combos5[i][2], combos5[i][3],combos5[i][4],
             Paulify(symp_mul(symplectify(combos5[i][1]), symplectify(combos5[i][2])))]
    
    combo2 = combo.copy()
    combo2.sort()
    
    if tuple(combo2) not in combos6prime:
        combos6prime.append(tuple(combo2))
        combos6.append(tuple(combo))
        
print("Done")

In [ ]:
len(combos6)

---

Finally, generating the possible sets $\{g_1,g_2,g_3,g_1g_2,g_1g_3,g_2g_3,g_1g_2g_3\}$. aka the final goal, and removing the copies (and the sets that have produced the identity matrix in the process) reduces the list to $1,080$.

In [ ]:
combos7 = []
combos7prime = []
for i in range(len(combos6)):
    combo = [combos6[i][0], combos6[i][1], combos6[i][2], combos6[i][3], combos6[i][4], combos6[i][5],
             Paulify(symp_mul(symplectify(combos6[i][2]), symplectify(combos6[i][3])))]
    
    if "-III" in tuple(combo):
        continue
    if "III" in tuple(combo):
        continue
    
    combo2 = combo.copy()
    combo2.sort()
    
    if tuple(combo2) not in combos7prime:
        combos7prime.append(tuple(combo2))
        combos7.append(tuple(combo))
        
for j in combos7:
    print(j)

In [ ]:
len(combos7)

### Conclusion of the Counting

We have managed to whittle all possible combinations of gates from $2,097,152$ down to our preferred $1,080$. 

---

### The Next Step

Now that we have the list of combos for $n=3$, we can find each corresponding $1080$ stabilizer states. To do this we must actually calculate the density matrix $\rho_{\left|\psi\right\rangle}$ via the formula:

$$\rho_{\left|\psi\right\rangle} = \frac{1}{8}\left(\mathbf{I} + g_1 + g_2 + g_3 + g_1g_2 + g_1g_3 + g_2g_3 + g_1g_2g_3\right)$$

Once we have a density matrix we can calculate the related stabilizer state. This is done by finding the eigenvalues of the $8\times 8$ density matrix. Of the eight eigenvalues found there will only be one non-zero eigenvalue. The eigenvector corresponding to the non-zero eigenvalue is the stabilizer state.

The below codeblock will calculate each of the $1080$ density matrices, find the non-zero eigenvalue and output the stabilizer state. In the future we can eventually try match the stabilizer states generated through the symplectic form to those through the affine form and find the pattern that links the two.

In [ ]:
gate_dict = gate_dictmaker(3)

In [ ]:
#this just numbers the states for my own sanity
i = 0

vect_list = []

#run over all the combos
for combo in combos7:
    i+=1
    print("_________________")
    
    #print the combo
    print(i,")", combo)
    
    #calculate and print the matrix
    
    #mat = gate_dict["III"] #+ gate_dict[combo[0]] + gate_dict[combo[1]] + gate_dict[combo[2]] + gate_dict[combo[3]]
    #for gate in combo:
    #    mat += gate_dict[gate]
    mat = gate_dict["III"] + gate_dict[combo[0]] + gate_dict[combo[1]] + gate_dict[combo[2]] + gate_dict[combo[3]]
    mat2 = mat + gate_dict[combo[4]] + gate_dict[combo[5]] + gate_dict[combo[6]]
    print(mat2)
    
    #calculate the e.vects and e.vals associated with the matrix
    w,v=eig(mat2)
    
    #print(np.round(w,2))
    #print(np.round(v,2))
    #find the e.vect associated with the non-zero e.val and print it
    print(np.round([v[0][np.argmax(w)], v[1][np.argmax(w)], v[2][np.argmax(w)], v[3][np.argmax(w)], v[4][np.argmax(w)], v[5][np.argmax(w)], v[6][np.argmax(w)], v[7][np.argmax(w)]],2))
    print("")